<font color='green' size='6'>**Домашняя работа 1**</font>  
<font color='#11a642' size='5'>**Тема: Машинное обучение**</font>  
<font size='5'>**Задание:**</font>  

Домашняя работа основана на данных о стоимости недвижимости.

**Описание данных:**

| Признак     | Описание                                                 |
|-------------|-----------------------------------------------------------|
| MedInc      | Средний доход домохозяйства (в десятках тысяч долларов)  |
| HouseAge    | Средний возраст домов в районе                           |
| AveRooms    | Среднее количество комнат на дом                         |
| AveBedrms   | Среднее количество спален на дом                         |
| Population  | Население района                                         |
| AveOccup    | Среднее количество человек на дом                        |

- target - стомиость недвижимости, что и нужно вам предсказать (это и есть целевой признак)

**Что нужно сделать:**

1. Проведите разведочный анализ данных (EDA)

2. Заполнить пропуски любыми 2 способами, что обсуждали на лекции. Объясните свой выбор

3. Построите модель линейной регрессии c помощью LinearRegression.

4. Оцените качество моделей. Выберите ту модель+метод заполнения пропусков, который даёт наилучший результат, и обоснуйте свой выбор.







# <font color='#11a642' size='6'> **Импорт и установка библиотек**

In [1]:
# %pip install scikit-learn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.impute import SimpleImputer


# <font color='#11a642' size='6'> **Загрузка данных**

In [2]:
# 1. Загрузка датасета
!gdown 1yQgwqFxwkHtZL2PZ2waF4Pg3Sb9hyHo9


Downloading...
From: https://drive.google.com/uc?id=1yQgwqFxwkHtZL2PZ2waF4Pg3Sb9hyHo9
To: c:\Users\alexander\Desktop\python_innopolis_ml\part_2\df_with_nan.csv

  0%|          | 0.00/1.16M [00:00<?, ?B/s]
 45%|████▌     | 524k/1.16M [00:00<00:00, 3.48MB/s]
100%|██████████| 1.16M/1.16M [00:00<00:00, 6.01MB/s]


In [3]:
df = pd.read_csv('df_with_nan.csv')

In [4]:
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Target
0,8.3252,41.0,6.984127,1.023810,322,2.555556,4.526
1,8.3014,21.0,6.238137,0.971880,2401,2.109842,3.585
2,7.2574,52.0,8.288136,1.073446,496,2.802260,3.521
3,5.6431,52.0,5.817352,1.073059,558,2.547945,3.413
4,3.8462,52.0,6.281853,1.081081,565,2.181467,3.422


In [5]:
# 2. Разделение признаков и целевой переменной
X = df.drop('Target', axis=1)
y = df['Target']

In [6]:
y.head()

0    4.526
1    3.585
2    3.521
3    3.413
4    3.422
Name: Target, dtype: float64

# <font color='#11a642' size='6'> **Разведочный анализ данных**

## <font color='#11a642' size='5'> Оцените размер вашего датасета


1.   Сколько ГБ ваш датасет
2.   Сколько строк и столбцов



In [7]:
# Сколько ГБ ваш датасет
# ваш код

print(f"Размер в памяти: {df.memory_usage(deep=True).sum() / (1024**3):.6f} ГБ")

Размер в памяти: 0.001077 ГБ


In [8]:
# Сколько строк и столбцов
# ваш код

print(f"Количество строк: {df.shape[0]:,}")
print(f"Количество столбцов: {df.shape[1]}")

Количество строк: 20,640
Количество столбцов: 7


## <font color='#11a642' size='5'> Анализ числовых (непрерывных) признаков


- для каждого признака посчитайте минимум, максимум, среднее, медиану

In [9]:
# ваш код

# Разделение на признаки и целевую переменную
X = df.drop('Target', axis=1)
y = df['Target']

stats_df = pd.DataFrame({
    'Признак': X.columns,
    'Минимум': X.min().values,
    'Максимум': X.max().values,
    'Среднее': X.mean().values,
    'Медиана': X.median().values,
    'Стд. отклонение': X.std().values
})
print(stats_df.round(3).to_string(index=False))

   Признак  Минимум  Максимум  Среднее  Медиана  Стд. отклонение
    MedInc    0.500    15.000    3.870    3.535            1.900
  HouseAge    1.000    52.000   28.628   29.000           12.578
  AveRooms    0.846   141.909    5.429    5.229            2.475
 AveBedrms    0.333    34.067    1.097    1.049            0.474
Population    3.000 35682.000 1425.477 1166.000         1132.462
  AveOccup    0.692  1243.333    3.071    2.818           10.386


## <font color='#11a642' size='5'> Анализ пропусков


In [10]:
# ваш код

missing_info = pd.DataFrame({
    'Признак': df.columns,
    'Количество пропусков': df.isnull().sum().values,
    'Процент пропусков': (df.isnull().sum() / len(df) * 100).values
})
missing_info = missing_info[missing_info['Количество пропусков'] > 0]
if len(missing_info) > 0:
    print(missing_info.round(2).to_string(index=False))
else:
    print("Пропусков не обнаружено!")

 Признак  Количество пропусков  Процент пропусков
  MedInc                    11               0.05
HouseAge                    11               0.05
AveRooms                    15               0.07


# <font color='#11a642' size='6'> **Разделите данные на трейн и тест**

In [11]:
# ваш код

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Размер обучающей выборки: {X_train.shape[0]:,} образцов")
print(f"Размер тестовой выборки: {X_test.shape[0]:,} образцов")

Размер обучающей выборки: 16,512 образцов
Размер тестовой выборки: 4,128 образцов


# <font color='#11a642' size='6'> **Подготовка данных к обучению**



1. Обработка пропусков любыми 2 способами, изученных на лекции




In [12]:
# ваш код

#  СПОСОБ 1: Заполнение средним значением
imputer_mean = SimpleImputer(strategy='mean')
X_train_mean = pd.DataFrame(
    imputer_mean.fit_transform(X_train),
    columns=X_train.columns,
    index=X_train.index
)
X_test_mean = pd.DataFrame(
    imputer_mean.transform(X_test),
    columns=X_test.columns,
    index=X_test.index
)

# СПОСОБ 2: Заполнение медианой

imputer_median = SimpleImputer(strategy='median')
X_train_median = pd.DataFrame(
    imputer_median.fit_transform(X_train),
    columns=X_train.columns,
    index=X_train.index
)
X_test_median = pd.DataFrame(
    imputer_median.transform(X_test),
    columns=X_test.columns,
    index=X_test.index
)


# <font color='#11a642' size='6'> **Построение моделей машинного обучения**

## <font color='#11a642' size='5'> Построение линейной регрессии с помощью LinearRegression


In [13]:
# ваш код

def evaluate_model(y_true, y_pred, model_name):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    return {
        'Модель': model_name,
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R²': r2
    }

results = []

lr_mean = LinearRegression()
lr_mean.fit(X_train_mean, y_train)
y_pred_mean = lr_mean.predict(X_test_mean)
results.append(evaluate_model(y_test, y_pred_mean, "LinearRegression + Mean"))

lr_median = LinearRegression()
lr_median.fit(X_train_median, y_train)
y_pred_median = lr_median.predict(X_test_median)
results.append(evaluate_model(y_test, y_pred_median, "LinearRegression + Median"))




### <font color='#11a642' size='4'> Оценка качества  всех моделей


In [14]:
# ваш код

results_df = pd.DataFrame(results)
print("РЕЗУЛЬТАТЫ ОЦЕНКИ МОДЕЛЕЙ:")
print(results_df.round(4).to_string(index=False))

# Определение лучшей модели
best_model_idx = results_df['R²'].idxmax()
best_model = results_df.iloc[best_model_idx]

print(f"ЛУЧШАЯ МОДЕЛЬ: {best_model['Модель']}")
print(f"R² Score: {best_model['R²']:.4f}")
print(f"RMSE: {best_model['RMSE']:.4f}")

РЕЗУЛЬТАТЫ ОЦЕНКИ МОДЕЛЕЙ:
                   Модель    MSE   RMSE    MAE     R²
  LinearRegression + Mean 0.6427 0.8017 0.5798 0.5096
LinearRegression + Median 0.6428 0.8017 0.5798 0.5095
ЛУЧШАЯ МОДЕЛЬ: LinearRegression + Mean
R² Score: 0.5096
RMSE: 0.8017


In [18]:
# Выбираем лучшую модель
if best_model['Модель'] == "LinearRegression + Mean":
    best_lr_model = lr_mean
    feature_names = X_train_mean.columns
else:
    best_lr_model = lr_median
    feature_names = X_train_median.columns

coefficients_df = pd.DataFrame({
    'Признак': feature_names,
    'Коэффициент': best_lr_model.coef_,
    'Важность': np.abs(best_lr_model.coef_)
}).sort_values('Важность', ascending=False)

print("\nКОЭФФИЦИЕНТЫ МОДЕЛИ:")
print("-" * 40)
print(coefficients_df.round(4).to_string(index=False))
print(f"\nСвободный член (intercept): {best_lr_model.intercept_:.4f}")


КОЭФФИЦИЕНТЫ МОДЕЛИ:
----------------------------------------
   Признак  Коэффициент  Важность
 AveBedrms       1.1092    1.1092
    MedInc       0.5455    0.5455
  AveRooms      -0.2227    0.2227
  HouseAge       0.0168    0.0168
  AveOccup      -0.0046    0.0046
Population       0.0000    0.0000

Свободный член (intercept): -0.5511


# <font color='#11a642' size='4'> Ваши выводы
- какую из моделей вы рекомендуете использовать для прогноза? Почему?



ВАШ ОТВЕТ

In [23]:
print(f"""
1. РАЗМЕР ДАННЫХ:
   • Датасет содержит {df.shape[0]:,} записей и {df.shape[1]} признаков
   • Размер в памяти: {df.memory_usage(deep=True).sum() / (1024**3):.6f} ГБ

2. ПРОПУСКИ В ДАННЫХ:
   • Обнаружены пропуски в {missing_info.shape[0] if len(missing_info) > 0 else 0} признаках
   • Применены два метода заполнения: среднее и медиана

3. КАЧЕСТВО МОДЕЛЕЙ:
   • Лучшая модель: {best_model['Модель']}
   • R² Score: {best_model['R²']:.4f} (объясняет {best_model['R²']*100:.1f}% дисперсии)
   • RMSE: {best_model['RMSE']:.4f}

4. ВАЖНЫЕ ПРИЗНАКИ:
   • Наиболее влиятельный: {coefficients_df.iloc[0]['Признак']} (коэф. {coefficients_df.iloc[0]['Коэффициент']:.4f})
   • Наименее влиятельный: {coefficients_df.iloc[-1]['Признак']} (коэф. {coefficients_df.iloc[-1]['Коэффициент']:.4f})
""")

print("\nРЕКОМЕНДАЦИИ:")
print("-" * 20)

recommendation = "заполнение средним" if "Mean" in best_model['Модель'] else "заполнение медианой"
print(f"""
• Рекомендуется использовать модель LinearRegression с методом "{recommendation}"
• Причины выбора:
  - Показывает наилучшее качество по метрике R² ({best_model['R²']:.4f})
  - Имеет наименьшую ошибку RMSE ({best_model['RMSE']:.4f})
  - Простая и интерпретируемая модель
  
• Дальнейшие улучшения:
  - Можно попробовать полиномиальные признаки
  - Применить регуляризацию (Ridge, Lasso)
  - Использовать более сложные методы заполнения пропусков
""")


1. РАЗМЕР ДАННЫХ:
   • Датасет содержит 20,640 записей и 7 признаков
   • Размер в памяти: 0.001077 ГБ

2. ПРОПУСКИ В ДАННЫХ:
   • Обнаружены пропуски в 3 признаках
   • Применены два метода заполнения: среднее и медиана

3. КАЧЕСТВО МОДЕЛЕЙ:
   • Лучшая модель: LinearRegression + Mean
   • R² Score: 0.5096 (объясняет 51.0% дисперсии)
   • RMSE: 0.8017

4. ВАЖНЫЕ ПРИЗНАКИ:
   • Наиболее влиятельный: AveBedrms (коэф. 1.1092)
   • Наименее влиятельный: Population (коэф. 0.0000)


РЕКОМЕНДАЦИИ:
--------------------

• Рекомендуется использовать модель LinearRegression с методом "заполнение средним"
• Причины выбора:
  - Показывает наилучшее качество по метрике R² (0.5096)
  - Имеет наименьшую ошибку RMSE (0.8017)
  - Простая и интерпретируемая модель

• Дальнейшие улучшения:
  - Можно попробовать полиномиальные признаки
  - Применить регуляризацию (Ridge, Lasso)
  - Использовать более сложные методы заполнения пропусков

